# Numerical template

**Prerequisites:**

- This notebook must have been generated using the Gabarit's numerical template. 
- **Launch this notebook with a kernel using your project virtual environment**. In order to create a kernel linked to your virtual environment : `pip install ipykernel --user --name=your_venv_name` (once your virtual environment is activated). Obviously, the project must be installed on this virtual environment

---
---
---

## 1. How this template works

### Why use gabarit's numerical template ?

The numerical template automatically generates a project folder and python code containing mainstream models and facilitating their industrialization.

The generated project can be used for **classification** and **regression** tasks on numerical data. Of course, you have to adapt it to your particular use case. 

### Structure of the generated project

<table style="font-family: monospace;">
    <tr>
        <td style="border: none; padding: 0px;">.</td>
        <td style="border: none; padding: 0px;"></td>
        <td style="border: none; padding: 0px; color: green;"></td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">{{package_name}}</td>
        <td style="border: none; padding: 0px; color: green;"># The package</td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">│├──</td>
        <td style="border: none; padding: 0px;">models_training</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing all the modules related to the models</td>
    </tr>
        <td style="border: none; padding: 0px;">│├──</td>
        <td style="border: none; padding: 0px;">monitoring</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing all the modules related to the explainers and MLflow</td>
    </tr>
    </tr>
        <td style="border: none; padding: 0px;">│└──</td>
        <td style="border: none; padding: 0px;">preprocessing</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing all the modules related to the preprocessing</td>
    </tr>
    </tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">{{package_name}}-data</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing all the data (datasets, embeddings, etc.)</td>
    </tr>
    </tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">{{package_name}}-exploration</td>
        <td style="border: none; padding: 0px; color: green;"># Folder where all your experiments and explorations must go</td>
    </tr>
    </tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">{{package_name}}-models</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing all the generated models</td>
    </tr>
    </tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">{{package_name}}-pipelines</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing pipelines</td>
    </tr>
    </tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">{{package_name}}-ressources</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing some ressources such as the instructions to upload a model</td>
    </tr>
    </tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">{{package_name}}-scripts</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing examples script to preprocess data, train models, predict and use a demonstrator</td>
    </tr>
    </tr>
        <td style="border: none; padding: 0px;">│└──</td>
        <td style="border: none; padding: 0px;">utils</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing utils scripts (such as split train/test, sampling, etc...)</td>
    </tr>
    </tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">{{package_name}}-tutorial</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing notebook tutorials, including this one</td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">tests</td>
        <td style="border: none; padding: 0px; color: green;"># Folder containing all the unit tests</td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">.gitignore</td>
        <td style="border: none; padding: 0px; color: green;"></td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">.coveragerc</td>
        <td style="border: none; padding: 0px; color: green;"></td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">Makefile</td>
        <td style="border: none; padding: 0px; color: green;"></td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">nose_setup_coverage.cfg</td>
        <td style="border: none; padding: 0px; color: green;"></td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">README.md</td>
        <td style="border: none; padding: 0px; color: green;"></td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">requirements.txt</td>
        <td style="border: none; padding: 0px; color: green;"></td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">├──</td>
        <td style="border: none; padding: 0px;">setup.py</td>
        <td style="border: none; padding: 0px; color: green;"></td>
    </tr>
    <tr>
        <td style="border: none; padding: 0px;">└──</td>
        <td style="border: none; padding: 0px;">version.txt</td>
        <td style="border: none; padding: 0px; color: green;"></td>
    </tr>
</table>

### General principles on the generated packages

- Data must be saved in the `{{package_name}}-data` folder

- Trained models will automatically be saved in the `{{package_name}}-models` folder

- Be aware that all the functions/methods for writing/reading files uses these two folders as base. Thus when a script has an argument for the path of a file/model, the given path should be **relative** to the `{{package_name}}-data`/`{{package_name}}-models` folders.

- The provided scripts in `{{package_name}}-scripts` are given as example. You can use them to help you develop but their use is not required.

- You can use this package for mono-label and multi-labels tasks (`multi_label` argument in models' classes)

- The modelling part is structured as follows :
    - `ModelClass`: main class taking care of saving data and metrics (among other)
    - `ModelPipeline`: child class of ModelClass managing all models related to a sklearn pipeline
    - `ModelKeras`: child class of ModelClass managing all models using Keras
    - `ModelPyTorch`: child class of ModelClass managing all models using Torch (experimental)
    
---
---
---

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

sys.path.append(os.path.abspath(''))

from sklearn.datasets import load_wine
from {{package_name}}.utils import get_data_path
from tutorial_exercices import answers, verify, utils

# La fonction get_data_path rebviue 
DATA_PATH = get_data_path()

## 2. Use the template to train your first model

### Wine dataset
We are going to use generated python package in a classification problem. 

We are going to work with [Wine recognition dataset](https://archive.ics.uci.edu/ml/datasets/wine) from [sklearn](https://scikit-learn.org/stable/datasets/toy_dataset.html#wine-recognition-dataset).
This dataset result from a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars.

First we load the dataset thanks to `sklearn.datasets.load_wine` and save the dataset as a csv file in `{{package_name}}-data` :

In [ ]:
wine_dataset = load_wine(as_frame=True)

df_wine = wine_dataset["data"]
df_wine["target"] = wine_dataset["target"]

# Save dataset to {{package_name}}-data
DATA_PATH = get_data_path()
DATASET_WINE_FILENAME = "wine.csv"
DATASET_WINE_PATH = os.path.join(DATA_PATH, DATASET_WINE_FILENAME)

df_wine.to_csv(DATASET_WINE_PATH, sep=";", index=None)
print(f"Wine dataset saved to {DATASET_WINE_PATH}")

df_wine.head()

You can verify that a new file called `wine.csv` is present in your `{{package_name}}-data` directory. Notice the use of `get_data_path` function from `{{package_name}}.utils`. It returns the data folder.

### Exercice 1 : train / valid / test split

**Goal:**

- Split the main dataset in train / valid / test sets

**TODO:**
- Use the script `utils/0_split_train_valid_test.py` on the dataset `{{package_name}}-data/wine.csv`
- We want a 'random' split but **with a random seed set to 42** (in order to always reproduce the same results)
- We use the default splitting ratios (0.6 / 0.2 / 0.2)

**Help:**
- The file `utils/0_split_train_valid_test.py` splits a dataset in 3 .csv files:
    - `{filename}_train.csv`: the training dataset
    - `{filename}_valid.csv`: the validation dataset
    - `{filename}_test.csv`: the test dataset
- You can specify the type of split : random, stratified or hierarchical (here, use random)
- Reminder: the path to the file to process is relative to `{{package_name}}-data`
- To get the possible arguments of the script: `python 0_split_train_valid_test.py --help`
- Don't forget to activate your virtual environment ...

#### Exercice 1 : Verify your answer ✔

In [ ]:
verify.verify_exercice_1()

#### Exercice 1 : Solution 💡

In [ ]:
answers.answer_exercice_1()

### Exercice 2 : random sample

**Goal:**

- Get a random sample of the file `wine.csv` (n=10) (we won't use it, this exercise is just here to show what can be done)

**TODO:**
- Use the script `utils/0_create_samples.py` on the dataset `{{package_name}}-data/wine.csv`
- We want a sample of 10 lines

**Help:**
- The file `utils/0_create_samples.py` samples a dataset
- To get the possible arguments of the script: `python 0_create_samples.py --help`
- Don't forget to activate your virtual environment ...

#### Exercice 2 : Verify your answer ✔

In [ ]:
verify.verify_exercice_2()

#### Exercice 2 : Solution 💡

In [ ]:
answers.answer_exercice_2()

### Exercice 3 : pre-processing

- The script `1_preprocess_data.py` applies a preprocessing pipeline **to all columns of specified files except target columns**
- The argument `--target_cols` is used to specify which columns are targets in order to preserve these columns. Do not forget
to scpeify that `target` is the target column.
- It works as follows:
    - In `preprocessing/preprocess.py`: 
        - There is a dictionary of functions (`pipelines_dict`): key: str -> function 
            - /!\ Don't remove the default element 'no_preprocess': lambda x: x /!\ 
        - There are preprocessing functions
    - In `1_preprocess_data.py` :
        - We retrieve the dictionary of functions from `preprocessing/preprocess.py` 
        - If a `preprocessing` argument is specified, we keep only the corresponding key from the dictionnary 
        - Otherwise, we keep all keys (except `no_preprocess`) 
        - For each entry of the dictionary, we:
            - Get the associated preprocessing function
            - Load data
            - apply the preprocessing function
            - Save the result -> {file_name}_{key}.csv 
- To get the possible arguments of the script: `python 1_preprocess_data.py --help`
- By default the preprocessing pipeline used is `preprocess_P1`, you can see its content below
- Don't forget to activate your virtual environment ...

**Important:**
- Each preprocessed file is saved in the `{{package_name}}-data` folder.
- To track which preprocessed has been done, we add a first line to these files as a metadata line (e.g. `#preprocess_P1`).

In [ ]:
from {{package_name}}.preprocessing.preprocess import preprocess_P1

# Source code of preprocess_P1 function
utils.display_source(preprocess_P1)

As you can see, the default pipeline `preprocess_P1` works on numerical columns (keyword `num`), and apply `numeric_pipeline` to them which fill NA values with the median and apply a `StandardScaler`.

**Goal:**

- Apply the default preprocessing to `wine.csv`

**TODO:**
- Use the script `1_preprocess_data.py` on the dataset `{{package_name}}-data/wine.csv` to apply the default pipeline (`preprocess_P1`)
- The target column `target` should be preserved by the preprocessing script

#### Exercice 3 : Verify your answer ✔

In [ ]:
verify.verify_exercice_3()

#### Exercice 3 : Solution 💡

In [ ]:
answers.answer_exercice_3()

### Exercice 4 : custom pre-processing

**Goal:**

We are going to use a [`sklearn.preprocessing.MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler) instead of the [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) used in `preprocess_P1` to pre-process wine data

**TODO:**

- Create a `preprocess_P2` function in `{{package_name}}/preprocessing/preprocess.py` that use `MinMaxScaler` instead of `StandardScaler`
- Add `preprocess_P2` in `get_pipelines_dict` from `{{package_name}}/preprocessing/preprocess.py`
- Use the script `1_preprocess_data.py` on the dataset `{{package_name}}-data/wine.csv` to apply `preprocess_P2` pipeline

**Help:**

- You can use the source code of `preprocess_P1` seen in the previous exercice

#### Exercice 4 : Verify your answer ✔

In [ ]:
verify.verify_exercice_4()

#### Exercice 4 : Solution 💡

In [ ]:
answers.answer_exercice_4_preprocess_P2()

In [ ]:
answers.answer_exercice_4_preprocess_script()

### Exercice 5 : Pre-processing on train and validation data

**Goal:**

In the previous exercice we used `1_preprocess_data.py` to apply a preprocessing pipeline to `wine.csv`. To prevent [data leakage](https://machinelearningmastery.com/data-leakage-machine-learning/) we should preprocess training data separately from validation and test data. 

Each time `1_preprocess_data.py` is called, a new folder is created in `{{package_name}}-pipelines`. It contains a pipeline pickled object that can be used by `2_apply_existing_pipeling.py` script to apply a pipeline to a file.

We are going to use `1_preprocess_data.py` to apply default `preprocess_P1` to `wine_train.csv` and then apply the saved pipeline to `wine_valid.csv`. 

Note : Do not worry about applying `preprocess_P1` to `wine_test.csv`, it will be automatically when we will make predictions

**TODO:**

- Use the script `1_preprocess_data.py` to apply `preprocess_P1` pipeline on the dataset `wine_train.csv`
- Use the script `2_apply_existing_piepline.py` to apply saved `preprocess_P1` pipeline to `wine_valid.csv`


#### Exercice 5 : Verify your answer ✔

In [ ]:
verify.verify_exercice_5()

#### Exercice 5 : Solution 💡

In [ ]:
answers.answer_exercice_5()

### Exercice 6 : Train a model

**Goal:**

Train a classification model on preprocessed data. 

Default model in `3_training_classification.py` is [`ModelRidgeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html).

**TODO:**

- Use the script `3_training_classification.py` to train a `ModelRidgeClassifier` on `wine_train_preprocess_P1.csv`
- Use `wine_valid_preprocess_P1.csv` as validation dataset

**Help:**

- Use `3_training_classification.py -h` to see CLI helper.
- The script `3_training_classification.py` trains a model on a dataset
- It works as follows:
    - Read a train .csv file as input 
        - If a validation file is given, it will use it as validation data 
    - Manage `y_col` argument: 
        - If there is only one value, training in mono-label mode 
        - If several values, training in multi-labels mode 
    - **Manual modifications of the script**: 
        - **To change the model used** -> you have to comment/uncomment/modify the code in the "training" part (not necessary for this exercise) 
        - **To load datasets** -> if a dataset is not in the right format, you have to adapt the loading part (not necessary for this exercise) 
- Don't forget to activate your virtual environment ...
- If you get an `pkg_resources.DistributionNotFound`, you should try to install {{package_name}} : `python setup.py develop`

#### Exercice 6 : Verify your answer ✔

In [ ]:
verify.verify_exercice_6()

#### Exercice 6 : Solution 💡

In [ ]:
answers.answer_exercice_6()

### Exercice 7 : Try another classification model

The previous model already achieves a perfect score on validation dataset but the goal here is to see how to use a different kind of model for training.

**Goal:**

Train a `ModelGBTClassifier` classification model on preprocessed data. 

**TODO:**

- Change the script `3_training_classification.py` so it uses a `ModelGBTClassifier`
- Use the script `3_training_classification.py` to train `ModelGBTClassifier` a on `wine_train_preprocess_P1.csv` with `wine_valid_preprocess_P1.csv` as validation dataset

**Help:**

If you look at `3_training_classification.py` you will see that a lot of models are commented :

```python
model = model_ridge_classifier.ModelRidgeClassifier(
    x_col=x_col,
    y_col=y_col,
    level_save=level_save,
    preprocess_pipeline=preprocess_pipeline,
    ridge_params={"alpha": 1.0},
    multi_label=multi_label,
)
# model = model_logistic_regression_classifier.ModelLogisticRegressionClassifier(
#     x_col=x_col,
#     y_col=y_col,
#     level_save=level_save,
#     preprocess_pipeline=preprocess_pipeline,
#     lr_params={"penalty": "l2", "C": 1.0, "max_iter": 100},
#     multi_label=multi_label,
# )
#
# [...]
```
Comment and uncomment proper lines to use `ModelGBTClassifier` instead of `ModelRidgeClassifier`


#### Exercice 7 : Verify your answer ✔

In [ ]:
verify.verify_exercice_7()

#### Exercice 7 : Solution 💡

In [ ]:
answers.answer_exercice_7()

As you can see this model do not perform perfectly on validation dataset, probably due to overfitting. We are going to stick with our `ModelRidgeClassifier`

### Exercice 8 : Test ModelRidgeClassifier on test data

**Goal:**

Test your `ModelRidgeClassifier` model on test data. 

**TODO:**

- Use the script `4_predict.py` to make prediction about cultivars in test data.
- Use argument `[-y Y_COL [Y_COL ...]]` to obtain performance on test data.

**Help:**

- Use `4_predict.py -h` to see CLI helper.
- If you get an `ValueError: There are some missing mandatory columns`, you probably made predictions on a preprocessed version of `wine_test.csv` when you should have made predictions on `wine_test.csv`. This error is caused by the first row of the preprocessed file which contains metadata infos.

#### Exercice 8 : Verify your answer ✔

In [ ]:
verify.verify_exercice_8()

#### Exercice 8 : Solution 💡

In [ ]:
answers.answer_exercice_8()

---
---
---


## 3. Use a saved model in python

In this section, we will see how to load a saved model in python for use with new data

### Load a saved model

First choose one of your saved models :

In [ ]:
import os
from pathlib import Path
from {{package_name}}.utils import get_models_path
from {{package_name}}.models_training import utils_models

MODELS_PATH = Path(get_models_path())

# This line list saved model in {{package_name}}-models
saved_model_names = sorted([model.name for model in MODELS_PATH.glob("*/*")])
print("\n".join(saved_model_names))

Then load it with `utils_models.load_model` :

In [ ]:
model_name = saved_model_names[-1]
print(model_name)

model, model_conf = utils_models.load_model(model_name)

### Make predictions on new data

In [ ]:
from sklearn.datasets import load_wine
wine_dataset = load_wine(as_frame=True)

wine_dataset["data"].sample(3)

The `model` object as a `preprocess_pipeline` argument that can be used in combination to `utils_models.apply_pipeline` to apply the same preprocessing on the new data :

In [ ]:
wine_dataset_prep = utils_models.apply_pipeline(wine_dataset["data"], model.preprocess_pipeline)
wine_dataset_prep.sample(3)

We can then simply use `mdeol.predict` to make predictions : 

In [ ]:
predictions = model.predict(wine_dataset_prep).astype(int)

# Verifying accuracy :
accuracy = sum(predictions == wine_dataset["target"]) / predictions.shape[0]
print(f"Accuracy : {accuracy:.2%}")

---
---
---


## 4. BONUS : Start up a small web app to introduce your models 🚀 

You are now ready to demonstrate how good your models work. We implemented a default ***Streamlit*** app., let's try it !

```bash
# do not forget to activate your virtual environment
# source venv_num_template/bin/activate 

streamlit run {{package_name}}-scripts/5_demonstrator.py
```

It will start a Streamlit app on the default port (8501)

Visit [http://localhost:8501](http://localhost:8501) to see you demonstrator